# Análisis de Firmas Mutacionales con pyMut
Este notebook demuestra cómo usar **pyMut** para realizar análisis de firmas mutacionales de forma **simple y directa**.

pyMut proporciona una API de alto nivel que permite generar análisis completos con pocas líneas de código.

## ¿Qué son las firmas mutacionales?

Las firmas mutacionales son patrones característicos que revelan procesos mutagénicos específicos:
- Exposición al tabaco
- Daño por radiación UV  
- Defectos en reparación del ADN
- Procesos de envejecimiento

---


## 1. Configuración Simple


In [7]:
# Importaciones simples
import pandas as pd
import numpy as np
from pyMut import PyMutation
from pyMut.core import MutationMetadata

# Configurar para gráficos de alta calidad
PyMutation.configure_high_quality_plots()

print("✅ pyMut configurado correctamente")


✅ High-quality configuration activated for matplotlib
   • DPI: 300 (high resolution)
   • Margins: automatic (tight)
   • Format: optimized PNG
   ℹ️  Now all figures will be automatically saved in high quality
✅ pyMut configurado correctamente


## 2. Cargar Datos


In [9]:
# Cargar datos reales de TCGA LAML (formato wide ya procesado)
import os

# Ruta al archivo de datos TCGA
data_path = "../src/pyMut/data/examples/tcga_laml_converted.tsv"

# Verificar que el archivo existe
if not os.path.exists(data_path):
    raise FileNotFoundError(f"No se encontró el archivo de datos: {data_path}")

# Cargar datos reales
print("📁 Loading TCGA LAML data...")
datos = pd.read_csv(data_path, sep='\t')

# Obtener columnas de muestras (formato TCGA-AB-XXXX)
sample_columns = [col for col in datos.columns if col.startswith('TCGA-AB-')]
n_samples = len(sample_columns)

# Mostrar información básica de los datos
print(f"✅ Data loaded successfully!")
print(f"   • {len(datos):,} mutations")
print(f"   • {n_samples:,} unique samples")
print(f"   • {datos['Hugo_Symbol'].nunique():,} unique genes")

# Mostrar las primeras columnas para verificar el formato
print(f"\n📊 Data format: Wide format (samples as columns)")
print(f"   • VCF-like columns: CHROM, POS, REF, ALT, FILTER")
print(f"   • Sample columns: {sample_columns[:3]}... (showing first 3)")
print(f"   • Gene info: Hugo_Symbol, Variant_Classification")

print(f"\n🔍 First 3 rows (selected columns):")
preview_cols = ['CHROM', 'POS', 'REF', 'ALT'] + sample_columns[:2] + ['Hugo_Symbol', 'Variant_Classification']
print(datos[preview_cols].head(3))

# Crear objeto PyMutation con datos reales
metadata = MutationMetadata(
    source_format="TCGA_VCF_like", 
    file_path=data_path,
    filters=["PASS"], 
    fasta="GRCh37"
)

py_mut = PyMutation(datos, metadata)

print(f"\n🎯 PyMutation object created successfully with TCGA LAML data!")
print(f"   • Ready for mutational signature analysis")
print(f"   • Data format: Wide (samples as columns)")
print(f"   • Total samples: {n_samples}")


📁 Loading TCGA LAML data...
✅ Data loaded successfully!
   • 2,207 mutations
   • 193 unique samples
   • 1,611 unique genes

📊 Data format: Wide format (samples as columns)
   • VCF-like columns: CHROM, POS, REF, ALT, FILTER
   • Sample columns: ['TCGA-AB-2988', 'TCGA-AB-2869', 'TCGA-AB-3009']... (showing first 3)
   • Gene info: Hugo_Symbol, Variant_Classification

🔍 First 3 rows (selected columns):
  CHROM        POS REF ALT TCGA-AB-2988 TCGA-AB-2869 Hugo_Symbol  \
0    17   67170916   T   C          T|C          T|T      ABCA10   
1     1   94490593   C   T          C|C          C|T       ABCA4   
2     2  169780249   G   A          G|G          G|G      ABCB11   

  Variant_Classification  
0            SPLICE_SITE  
1      MISSENSE_MUTATION  
2      MISSENSE_MUTATION  

🎯 PyMutation object created successfully with TCGA LAML data!
   • Ready for mutational signature analysis
   • Data format: Wide (samples as columns)
   • Total samples: 193


## 3. Análisis Completo

**¡Esta es la potencia de pyMut!** Una sola llamada genera todo el análisis:

- Extracción automática de la matriz de 96 categorías
- Factorización NMF para identificar firmas
- Perfiles de firmas mutacionales
- Similitud con firmas COSMIC conocidas  
- Mapas de calor de contribuciones
- Gráficos de barras y donas


In [10]:
fig_completo = py_mut.mutational_signature_plot(
    n_signatures=3,
    title="TCGA LAML - Mutational Signature Analysis",
    figsize=(20, 24),
    show_interactive=False
)

# Guardar el análisis completo
fig_completo.savefig("analisis_mutational_signature_completo.png")
print("💾 Analysis saved as: analisis_mutational_signature_completo.png")


Extracting mutation matrix...
Matrix shape: (0, 96)
Total mutations: 0
💾 Analysis saved as: analisis_mutational_signature_completo.png


<Figure size 6000x7200 with 0 Axes>

## 4. Análisis Específicos (Opcional)

Si necesitas solo una visualización específica, pyMut también lo permite:


In [11]:
# Si solo necesitas una visualización específica, puedes usar las funciones individuales
from pyMut.visualizations.mutational_signature import (
    extract_mutation_matrix,
    perform_nmf, 
    create_signature_profile_plot
)

# Ejemplo de perfiles de firmas mutacionales únicamente
matriz, muestras = extract_mutation_matrix(datos)
W, H = perform_nmf(matriz, n_signatures=3)

fig_perfiles = create_signature_profile_plot(
    H, 
    signature_names=["Signature 1", "Signature 2", "Signature 3"],
    figsize=(20, 6)
)

print("📊 Individual signature profiles generated")
fig_perfiles.savefig("perfiles_firmas_mutacionales.png")
print("💾 Signature profiles saved as: perfiles_firmas_mutacionales.png")

ValueError: Found array with 0 sample(s) (shape=(0, 96)) while a minimum of 1 is required by NMF.